## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Modelado y evaluación

---

En esta libreta se realiza la experimentación para generación del modelo de predicción objetivo del proyecto y la evaluación del mismo.
La versión del dataset a utilizar es la obtenida a partir de las operaciones de transformación.

---

En esta versión de la libreta se va a incorporar el registro de los detalles de la experimentación con la librería MFLOW.

In [25]:
# Se importan las librerías necesarias y se suprimen las advertencias
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=UserWarning)

Se agrega la librería mlflow y se configura inicialmente

In [26]:
import mlflow
import mlflow.sklearn
import datetime as datetime

# Configuración de MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("Proyecto 13MBID-Abr2526 - Experimentacion Original")

<Experiment: artifact_location=('file:d:/Formación/Master en Big Data y Ciencia de '
 'Datos/Asignaturas/13_Metodologias de gestion y diseño proyectos Big '
 'Data/Template '
 'Original/Master_VIU_13MBID_Abr2526/notebooks/mlruns/211776156448520195'), creation_time=1762503407948, experiment_id='211776156448520195', last_update_time=1762503407948, lifecycle_stage='active', name='Proyecto 13MBID-Abr2526 - Experimentacion Original', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [27]:
# Lectura de los datos
df = pd.read_csv('../data/processed/bank_processed.csv')
df.head(5)

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,Teléfono,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,no,no,Teléfono,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,yes,no,Teléfono,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,Teléfono,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,yes,Teléfono,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [28]:
# Se divide el dataset en variables predictoras y variable objetivo
X = df.drop('y', axis=1)
y = df['y']

In [29]:
# Se genera el conjunto de entrenamiento y prueba con estratificación
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [31]:
# Se separan las columnas numéricas
numerical_columns=X_train.select_dtypes(exclude='object').columns
display(numerical_columns)

categorical_columns=X_train.select_dtypes(include='object').columns
display(categorical_columns)

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')

Index(['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month',
       'day_of_week', 'poutcome'],
      dtype='object')

In [32]:
# Se verifica la distribución de la variable objetivo en el conjunto de entrenamiento
y_train.value_counts()

y
0    27179
1     3406
Name: count, dtype: int64

In [33]:
# Se crea un pipeline para preprocesamiento de datos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler  

# Pipeline para valores numéricos
num_pipeline = Pipeline(steps=[
    ('RobustScaler', RobustScaler())
])

# Pipeline para valores categóricos
cat_pipeline = Pipeline(steps=[
    ('OneHotEncoder', OneHotEncoder(drop='first',sparse_output=False))
])

# Se configuran los preprocesadores
preprocessor_full = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
]).set_output(transform='pandas')

In [34]:
preprocessor_train_valid = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
]).set_output(transform='pandas')

In [35]:
# Se ajusta y transforma el conjunto de entrenamiento y prueba
x_train_prep = preprocessor_full.fit_transform(X_train)
x_test_prep = preprocessor_full.transform(X_test)

In [37]:
# Se aplica submuestreo a los datos preprocesados
from sklearn.utils import resample

# Combinar los datos preprocesados con las etiquetas
train_data = x_train_prep.copy()
train_data['target'] = y_train.reset_index(drop=True)

# Separar por clase
class_0 = train_data[train_data['target'] == 0]
class_1 = train_data[train_data['target'] == 1]

# Encontrar la clase minoritaria
min_count = min(len(class_0), len(class_1))

# Submuestreo balanceado - tomar una muestra igual al tamaño de la clase minoritaria
class_0_balanced = resample(class_0, n_samples=min_count, random_state=42)
class_1_balanced = resample(class_1, n_samples=min_count, random_state=42)

# Combinar las clases balanceadas
balanced_data = pd.concat([class_0_balanced, class_1_balanced])

# Separar características y objetivo
x_train_resampled = balanced_data.drop('target', axis=1)
y_train_resampled = balanced_data['target']

print(f"Tamaño original: {len(x_train_prep)}")
print(f"Tamaño balanceado: {len(x_train_resampled)}")
print(f"Distribución balanceada: {y_train_resampled.value_counts()}")

Tamaño original: 30585
Tamaño balanceado: 5438
Distribución balanceada: target
0.0    2719
1.0    2719
Name: count, dtype: int64


--- 
Este apartado se va a cambiar

In [ ]:
from sklearn.model_selection import cross_val_score


# Se genera una función para realizar validación cruzada
def cross_val(model):
    scores = cross_val_score(model,x_train_resampled , y_train_resampled, cv=5, scoring='f1')
    print('cross validation f1 scores',scores*100)
    print('cross validation f1 mean',scores.mean()*100)
    print('cross validation f1 std',scores.std())
    print('-'*50)
    scores = cross_val_score(model,x_train_resampled , y_train_resampled, cv=5, scoring='recall')
    print('cross validation recall scores',scores*100)
    print('cross validation recall mean',scores.mean()*100)
    print('cross validation recall std',scores.std())

---
El cambio consiste en que la función directamente registre los detalles de la experimentación en mlflow.

In [38]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from mlflow.models import infer_signature

def cross_val_mlflow(model, model_name, params=None):
    """
    Realiza validación cruzada y registra los resultados en MLflow.
    """
    with mlflow.start_run(run_name=model_name):
        f1_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='f1')
        f1_mean = f1_scores.mean()
        f1_std = f1_scores.std()

        recall_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='recall')
        recall_mean = recall_scores.mean()
        recall_std = recall_scores.std()

        precision_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='precision')
        precision_mean = precision_scores.mean()
        precision_std = precision_scores.std()

        accuracy_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
        accuracy_mean = accuracy_scores.mean()
        accuracy_std = accuracy_scores.std()
        
        # Entrenamos al modelo
        model.fit(x_train_resampled, y_train_resampled)

        # Hacemos predicciones
        y_pred = model.predict(x_test_prep)

        # Obtenemos model_signature
        signature = infer_signature(x_train_resampled, y_pred)

        test_f1 = f1_score(y_test, y_pred)
        test_recall = recall_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred)
        test_accuracy = accuracy_score(y_test, y_pred)

        # Registramos los parámetros y métricas en MLflow
        if params:
            mlflow.log_params(params)
        else:
            mlflow.log_params(model.get_params())

        mlflow.log_params({
            "train_samples": len(x_train_resampled),
            "test_samples": len(x_test_prep),
            "balancing_method": "undersampling",
            "cv_folds": 5
        })

        # Registramos las métricas de validación cruzada
        mlflow.log_metrics({
            "cv_f1_mean": f1_mean,
            "cv_f1_std": f1_std,
            "cv_recall_mean": recall_mean,
            "cv_recall_std": recall_std,
            "cv_precision_mean": precision_mean,
            "cv_precision_std": precision_std,
            "cv_accuracy_mean": accuracy_mean,
            "cv_accuracy_std": accuracy_std,
        })
        
        mlflow.log_metrics({  
            "test_f1": test_f1,
            "test_recall": test_recall,
            "test_precision": test_precision,
            "test_accuracy": test_accuracy
        })

        # Registramos el modelo
        mlflow.sklearn.log_model(
            model,
            name="model",
            signature=signature
        )

        print(f"Modelo {model_name} registrado en MLflow con ID de ejecución: {mlflow.active_run().info.run_id}")

        return model, {
            "cv_f1_mean": f1_mean,
            "cv_recall_mean": recall_mean,
            "cv_precision_mean": precision_mean,
            "cv_accuracy_mean": accuracy_mean,
            "cv_accuracy_std": accuracy_std,
            "test_f1": test_f1,
            "test_recall": test_recall,
            "test_precision": test_precision,
            "test_accuracy": test_accuracy
        }


---
Se cambian estas celdas por una sola que hace todas las llamadas

In [35]:
# Se aplica un modelo de regresión logística
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1,penalty='l2',solver='liblinear',random_state=1,max_iter=100,tol=0.000000001)

# cross validation scores
cross_val(lr)

NameError: name 'cross_val' is not defined

In [ ]:
# LinearSVC
from sklearn.svm import LinearSVC
svc = LinearSVC(max_iter=10000,tol=0.001)

# cross validation scores
cross_val(svc)

In [ ]:
# knclassifier
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=7)

# cross validation scores
cross_val(knc)

In [ ]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()

# cross validation scores
cross_val(tree)

In [ ]:
# decision tree plot
from sklearn.tree import plot_tree
tree.fit(x_train_prep, y_train)
plot_tree(tree, filled=True, rounded=True,max_depth=2,fontsize=10)
plt.show()

In [ ]:
# Se obtiene la matriz de confusión para el modelo
from sklearn.metrics import confusion_matrix

y_pred = tree.predict(x_test_prep)
cm = confusion_matrix(y_test, y_pred)
print(cm)


In [ ]:
# Se visualiza la matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no', 'yes'])
disp.plot()

---
Esta es la nueva celda que hace la invocación al proceso con mlflow

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import sklearn.naive_bayes as nb

# Resultados
resultados = {}
    
# Método 1 - Regresión Logística
lr = LogisticRegression(C=1,penalty='l2',solver='liblinear',random_state=17,max_iter=100,tol=0.000000001)
model_lr, metrics_lr = cross_val_mlflow(lr, "Logistic Regression")
resultados['Logistic Regression'] = metrics_lr

# Método 2 - SVC
svc = LinearSVC(max_iter=10000,tol=0.001, random_state=17)   
model_svc, metrics_svc = cross_val_mlflow(svc, "Linear SVC")
resultados['Linear SVC'] = metrics_svc
# Método 3 - KNN
knc = KNeighborsClassifier(n_neighbors=7)
model_knc, metrics_knc = cross_val_mlflow(knc, "K-Nearest Neighbors")
resultados['K-Nearest Neighbors'] = metrics_knc

# Método 4 - Decision Tree
tree = DecisionTreeClassifier(random_state=17)
model_tree, metrics_tree = cross_val_mlflow(tree, "Decision Tree Classifier")
resultados['Decision Tree Classifier'] = metrics_tree

# Método 5 - Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=17)   
model_rf, metrics_rf = cross_val_mlflow(rf, "Random Forest Classifier")
resultados['Random Forest Classifier'] = metrics_rf

# Método 6 - Naive Bayes
gnb = nb.GaussianNB()
model_gnb, metrics_gnb = cross_val_mlflow(gnb, "Gaussian Naive Bayes")
resultados['Gaussian Naive Bayes'] = metrics_gnb


Modelo Logistic Regression registrado en MLflow con ID de ejecución: cd1d78e1554b44c3a9121ee8bf0c4697
Modelo Linear SVC registrado en MLflow con ID de ejecución: 320688ccfaf949cb878c74deda281a22
Modelo K-Nearest Neighbors registrado en MLflow con ID de ejecución: 007ac4a682ae4bb58f1a9be01b901213
Modelo Decision Tree Classifier registrado en MLflow con ID de ejecución: 3e1d9fe537284db6848ca21b3de75b37
Modelo Random Forest Classifier registrado en MLflow con ID de ejecución: 9a1b4601423e4a0089d21d0235e1ae0a
Modelo Gaussian Naive Bayes registrado en MLflow con ID de ejecución: c68c5ca1d9a84a02838cb8671802924c


--- 
A continuación se realiza una comparación de los modelos para seleccionar el que va a ser utilizado posteriormente:

In [54]:
df_comparacion = pd.DataFrame(resultados).T
df_comparacion = df_comparacion.round(4)
df_comparacion = df_comparacion.sort_values(by='test_f1', ascending=False)

print(df_comparacion)

print("\nEl mejor modelo basado en F1 en el conjunto de pruebas es:", df_comparacion.index[0])
print("Valor de F1 en test:", df_comparacion.iloc[0]['test_f1'])
print("Valor de Recall en test:", df_comparacion.iloc[0]['test_recall'])


                          cv_f1_mean  cv_recall_mean  cv_precision_mean  \
Decision Tree Classifier      0.6992          0.7437             0.6598   
K-Nearest Neighbors           0.5699          0.5815             0.5591   
Gaussian Naive Bayes          0.6578          0.9507             0.5029   
Random Forest Classifier      0.6964          0.6925             0.7003   
Logistic Regression           0.5244          0.5204             0.5289   
Linear SVC                    0.5240          0.5204             0.5281   

                          cv_accuracy_mean  cv_accuracy_std  test_f1  \
Decision Tree Classifier            0.6800           0.0100   0.1657   
K-Nearest Neighbors                 0.5612           0.0137   0.1551   
Gaussian Naive Bayes                0.5055           0.0070   0.1469   
Random Forest Classifier            0.6980           0.0097   0.1224   
Logistic Regression                 0.5278           0.0213   0.1100   
Linear SVC                          0.5270

#### Predicción con datos nuevos (sin clasificar)

In [ ]:
df_nuevos = pd.read_csv('../data/raw/bank-additional-new.csv')
df_nuevos.head(5)

In [ ]:
# Diagnosticar el problema con los nuevos datos
print("Información del conjunto de datos nuevos:")
print(f"Forma: {df_nuevos.shape}")
print("\nTipos de datos:")
print(df_nuevos.dtypes)
print("\nValores nulos:")
print(df_nuevos.isnull().sum())
print("\nColumnas categóricas en nuevos datos:")
print(df_nuevos.select_dtypes(include='object').columns.tolist())
print("\nColumnas numéricas en nuevos datos:")
print(df_nuevos.select_dtypes(exclude='object').columns.tolist())

In [ ]:
# Comparar con los datos de entrenamiento originales
print("Comparación de columnas:")
print(f"Columnas en datos originales: {list(X.columns)}")
print(f"Columnas en datos nuevos: {list(df_nuevos.columns)}")

print("\nColumnas que están en nuevos pero no en originales:")
new_cols = set(df_nuevos.columns) - set(X.columns)
print(new_cols)

print("\nColumnas que están en originales pero no en nuevos:")
missing_cols = set(X.columns) - set(df_nuevos.columns)
print(missing_cols)

In [ ]:
# Se hace la predicción con los nuevos datos
# Primero, eliminar la columna objetivo si existe y preparar las características
X_new = df_nuevos.drop('y', axis=1) if 'y' in df_nuevos.columns else df_nuevos.copy()

# Asegurar que las columnas estén en el mismo orden que en el entrenamiento
X_new = X_new[X.columns]

# Manejar la columna contacted_before para que coincida con el formato de entrenamiento
# En entrenamiento: 'no', 'yes' (string)
# En nuevos datos: NaN -> necesita convertirse a 'no' (asumiendo que NaN significa no contactado)
X_new['contacted_before'] = X_new['contacted_before'].fillna('no')

# Convertir cualquier valor numérico a string si es necesario
if X_new['contacted_before'].dtype in ['float64', 'int64']:
    X_new['contacted_before'] = X_new['contacted_before'].map({0.0: 'no', 1.0: 'yes'}).fillna('no')

# Asegurar que contacted_before sea de tipo object como en entrenamiento
X_new['contacted_before'] = X_new['contacted_before'].astype('object')

# Transformar los nuevos datos usando el mismo preprocesador y predecir
try:
    x_new_prep = preprocessor_full.transform(X_new)
    
    y_new_pred = tree.predict(x_new_prep)
    print(f"\nPredicciones: {y_new_pred}")
    
    predictions_df = pd.DataFrame({
        'Cliente': range(1, len(y_new_pred) + 1),
        'Predicción_Numérica': y_new_pred,
        'Suscribirá': ['No' if pred == 0 else 'Sí' for pred in y_new_pred]
    })
    print("\nResultados detallados:")
    print(predictions_df.to_string(index=False))
    
    # Resumen de predicciones
    pred_counts = pd.Series(y_new_pred).value_counts()
    print("\nResumen de predicciones:")
    for pred_val, count in pred_counts.items():
        label = 'No realizará un depósito' if pred_val == 0 else 'Sí realizará un depósito'
        print(f"  {label}: {count} clientes ({count/len(y_new_pred)*100:.1f}%)")
    
except Exception as e:
    print(f"Error durante el preprocesamiento o predicción: {e}")
    print("Información adicional para depuración:")
    print(f"Tipos de datos en X_new:\n{X_new.dtypes}")